In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns,plotly.express as px



In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [3]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [5]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [6]:
df.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [7]:
df.shape

(5110, 12)

In [8]:
# drop the column id as it does not hold weight in the analysis
df.drop('id', axis = 1, inplace = True)
#fill missing values in feature bmi with mean
df.bmi.fillna(value=df.bmi.mean(), inplace=True)
#separating the features from the target column
X = df.drop('stroke', axis = 1)
y = df.stroke
#dummy encoding the categorical and discrete features for efficient deep learning
x = pd.get_dummies(X, columns = ['gender',  'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 
       'smoking_status'])
#splitting the data into train and test sets in the ratio 8:2
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
#preprocessing the data using the Robust Scaler
scaler = preprocessing.RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# defining and fitting the neural network model
model = keras.Sequential()
model.add(keras.layers.Dense(1000, activation = 'relu', input_dim = x.shape[1]))
model.add(keras.layers.Dense(1000, activation = 'relu'))
model.add(keras.layers.Dense(1000, activation = 'relu'))
model.add(keras.layers.Dense(1000, activation = 'relu'))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))
#compiling the model
model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

#fitting the train sets to train the model
model.fit(X_train,y_train.to_numpy(), batch_size = 20, epochs = 10, verbose = 1)
#predicting the target using the X_test data
y_pred = model.predict_classes(X_test)
print(y_pred.max(), y_pred.min())
#evaluate the model accuracy 
model.evaluate(X_test,y_test.to_numpy())
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

(4088, 23) (4088,)
(1022, 23) (1022,)
Epoch 1/10
205/205 [==============================] - 32s 108ms/step - loss: 0.2158 - accuracy: 0.9404
Epoch 2/10
205/205 [==============================] - 22s 108ms/step - loss: 0.1668 - accuracy: 0.9538
Epoch 3/10
205/205 [==============================] - 23s 113ms/step - loss: 0.1527 - accuracy: 0.9579
Epoch 4/10
205/205 [==============================] - 22s 109ms/step - loss: 0.1494 - accuracy: 0.9526s - loss: 0.1494 - accuracy: 
Epoch 5/10
205/205 [==============================] - 21s 104ms/step - loss: 0.1560 - accuracy: 0.9508
Epoch 6/10
205/205 [==============================] - 22s 105ms/step - loss: 0.1441 - accuracy: 0.9555s - loss: 0.144
Epoch 7/10
205/205 [==============================] - 23s 110ms/step - loss: 0.1449 - accuracy: 0.9521
Epoch 8/10
205/205 [==============================] - 23s 111ms/step - loss: 0.1299 - accuracy: 0.9566
Epoch 9/10
205/205 [==============================] - 22s 110ms/step - loss: 0.1151 - accuracy

C:\Users\SKETCH3D\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


1 0
32/32 [==============================] - 2s 40ms/step - loss: 0.2149 - accuracy: 0.9384
0.9383561643835616
[[959   1]
 [ 62   0]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       960
           1       0.00      0.00      0.00        62

    accuracy                           0.94      1022
   macro avg       0.47      0.50      0.48      1022
weighted avg       0.88      0.94      0.91      1022

